In [8]:
import pandas as pd
import numpy as np
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [9]:
dt = joblib.load("../result/PT dataset features label.joblib")

In [10]:
filtered_df = dt[ dt.label.isin(["A", "C"]) ] 
filtered_df["numeric_label"] = filtered_df.label.apply(lambda x: 0 if x=='A' else 1 if x=='C' else None)

C:\Users\Kian\AppData\Local\Temp\ipykernel_13824\1446778592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["numeric_label"] = filtered_df.label.apply(lambda x: 0 if x=='A' else 1 if x=='C' else None)


In [5]:
filtered_df = filtered_df.sample(frac=1)

In [7]:
filtered_df.head(2)

,Fp1_Delta_rel,Fp1_Theta_rel,Fp1_Alpha_rel,Fp1_Sigma_rel,Fp1_Beta_rel,Fp1_Gamma_rel,Fp2_Delta_rel,Fp2_Theta_rel,Fp2_Alpha_rel,Fp2_Sigma_rel,...,Cz_Gamma_rel,Pz_Delta_rel,Pz_Theta_rel,Pz_Alpha_rel,Pz_Sigma_rel,Pz_Beta_rel,Pz_Gamma_rel,group,label,numeric_label
12643,0.9584359662715475,0.02139107542285754,0.008016072466126397,0.0032257992625293747,0.006566514998219926,0.002364571578719362,0.9560117032763731,0.02349162525715205,0.007807682803465128,0.0032779950886563064,...,0.0042810739476969685,0.8368383647573424,0.05340677735312753,0.08105681524464484,0.008625300229815834,0.014948916641282733,0.005123825773786466,sub-049,C,1
9443,0.9654205558423754,0.019684052027146078,0.006472477939750562,0.0019026505190689299,0.0053274046108602955,0.0011928590607987954,0.9656190963999445,0.02097733015159907,0.005505444250434178,0.0021913123899859398,...,0.0009324848372955078,0.9263567076669021,0.027546618876392388,0.03931872455589075,0.0024106979936945115,0.003534700309697054,0.0008325505974227667,sub-037,C,1


In [34]:
features = filtered_df.iloc[:, :-3].to_numpy()
labels = filtered_df.numeric_label.to_numpy()
groups = filtered_df.group.to_numpy()

In [35]:
features.shape, labels.shape, groups.shape

((16410, 114), (16410,), (16410,))

In [46]:
%%time

loso = LeaveOneGroupOut()
feature_selection = True
use_feature_scaler = False
lasso_cv = LassoCV()

result = {
    "tr_label": [],
    "ts_label": [],
    
    "tr_predict": [],
    "ts_predict": [],
    
    "loso_tr_label": [],
    "loso_ts_label": [],
    
    "loso_tr_predict": [],
    "loso_ts_predict": [],
    
    "loso_tr_accuracy": [],
    "loso_ts_accuracy": [],
    
    "loso_tr_f1": [],
    "loso_ts_f1": [],
    
    "loso_tr_sensitivity": [],
    "loso_ts_sensitivity": [],
    
    "loso_tr_specificity": [],
    "loso_ts_specificity": []
}


for i, (tr_idx, ts_idx) in enumerate(loso.split(X=features, y=labels, groups=groups)):

    print(f"""### {i} subject compelete.
        test group: {np.unique(groups[ts_idx])}        
    """)
    
    clf = RandomForestClassifier()
    # clf = SVC()
    # clf = lda()
    
    tr_data, tr_label = features[tr_idx], labels[tr_idx]
    ts_data, ts_label = features[ts_idx], labels[ts_idx]

    if use_feature_scaler:
        scaler = StandardScaler()
        tr_data = scaler.fit_transform(tr_data)
        ts_data = scaler.transform(ts_data)
    
    if feature_selection:
        sfm = SelectFromModel(lasso_cv)
        tr_data = sfm.fit_transform(tr_data, tr_label)
        ts_data = sfm.transform(ts_data)

    clf.fit(tr_data, tr_label)

    result["tr_label"].extend(tr_label.tolist())
    result["ts_label"].extend(ts_label.tolist())
    
    result["tr_predict"].extend(clf.predict(tr_data).tolist())
    result["ts_predict"].extend(clf.predict(ts_data).tolist())
    
    result["loso_tr_label"].append(tr_label)
    result["loso_ts_label"].append(ts_label)
    
    result["loso_tr_predict"].append(clf.predict(tr_data))
    result["loso_ts_predict"].append(clf.predict(ts_data))
    
    result["loso_tr_accuracy"].append(accuracy_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_accuracy"].append(accuracy_score(ts_label, clf.predict(ts_data)))

    result["loso_tr_f1"].append(f1_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_f1"].append(f1_score(ts_label, clf.predict(ts_data)))
    
    result["loso_tr_sensitivity"].append(recall_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_sensitivity"].append(recall_score(ts_label, clf.predict(ts_data)))
    
    result["loso_tr_specificity"].append(precision_score(tr_label, clf.predict(tr_data)))
    result["loso_ts_specificity"].append(precision_score(ts_label, clf.predict(ts_data)))

    print(f"""
            ts acc: {result["loso_ts_accuracy"][-1]}
            ts f1: {result["loso_ts_f1"][-1]} 
            ts sensitivity: {result["loso_ts_sensitivity"][-1]} 
            ts specificity: {result["loso_ts_specificity"][-1]}
    """)
    
    # break

### 0 subject compelete.
        test group: ['sub-001']        
    


C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5402670186842897, tolerance: 0.31680906199084446
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.996484162757497, tolerance: 0.31680906199084446
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4233360035336773, tolerance: 0.3168220187757003
  model = cd_fast.enet_coordinate_descent_gram(



            ts acc: 0.9932885906040269
            ts f1: 0.0 
            ts sensitivity: 0.0 
            ts specificity: 0.0
    
### 1 subject compelete.
        test group: ['sub-002']        
    


C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.4438056178971692, tolerance: 0.31536692944114914
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.38835638321188526, tolerance: 0.31526714798626265
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3384935709329966, tolerance: 0.31526714798626265
  model = cd_fast.enet_coordinate_descent_gra


            ts acc: 0.47341772151898737
            ts f1: 0.0 
            ts sensitivity: 0.0 
            ts specificity: 0.0
    
### 2 subject compelete.
        test group: ['sub-003']        
    


...


            ts acc: 0.947103274559194
            ts f1: 0.9728331177231566 
            ts sensitivity: 0.947103274559194 
            ts specificity: 1.0
    
### 39 subject compelete.
        test group: ['sub-062']        
    


C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.608e+00, tolerance: 3.863e-01
  model = cd_fast.enet_coordinate_descent(



            ts acc: 0.9419642857142857
            ts f1: 0.9701149425287356 
            ts sensitivity: 0.9419642857142857 
            ts specificity: 1.0
    
### 40 subject compelete.
        test group: ['sub-063']        
    


C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.44686422709332874, tolerance: 0.3106150542671977
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8317928091846625, tolerance: 0.3106150542671977
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.31151828144402316, tolerance: 0.31052846099789133
  model = cd_fast.enet_coordinate_descent_gram


            ts acc: 0.4014962593516209
            ts f1: 0.5729537366548042 
            ts sensitivity: 0.4014962593516209 
            ts specificity: 1.0
    
CPU times: total: 1h 5min 53s
Wall time: 55min 44s


In [47]:
print(f"""
clf: {clf}
feature selection: {feature_selection}
feature selection model: {lasso_cv}
feature scaler: {use_feature_scaler}
ts acc average on loso: {np.mean(result["loso_ts_accuracy"])}
ts acc: {accuracy_score(result["ts_label"], result["ts_predict"])} 
ts f1: {f1_score(result["ts_label"], result["ts_predict"])} 
ts sensitivity: {recall_score(result["ts_label"], result["ts_predict"])} 
ts specificity: {precision_score(result["ts_label"], result["ts_predict"])}
""")


clf: RandomForestClassifier()
feature selection: True
feature selection model: LassoCV()
feature scaler: False
ts acc average on loso: 0.7836652730451434
ts acc: 0.7895795246800731 
ts f1: 0.7415999401332036 
ts sensitivity: 0.7076549557269352 
ts specificity: 0.7789655714510297



In [56]:
clf.predict(tr_data).shape

(16112,)

In [55]:
recall_score(ts_label, clf.predict(ts_data))

C:\Users\Kian\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.0

In [57]:
ts_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=i